# Tutorial Goal

This tutorial aims to show how to **configure and run** a predefined set of
**synthetic workload** using the **executor module** provided by LISA.

# Configure logging

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

In [2]:
# Execute this cell to enabled executor debugging statements
logging.getLogger('Executor').setLevel(logging.DEBUG)

# Target Configuration

In [3]:
from env import TestEnv

# Setup a test environment with target configuration
env = TestEnv({
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno',
    
    # Target board IP/MAC address
    "host"        : '192.168.0.1',
    
    # Login credentials
    "username"    : 'root',
    "password"    : 'test0000',

    # Folder where all the results will be collected
    "results_dir" : "ExecutorExample",
            
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "cpu_frequency",
         ],
         "buffsize" : 80 * 1024,
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'perf' ],
    
    # Modules required by these experiments
    "modules"     : [ 'bl', 'cpufreq' ],
})

2016-11-17 17:00:31,811 INFO    :         Target - Using base path: /home/bjackman/sources/lisa
2016-11-17 17:00:31,812 INFO    :         Target - Loading custom (inline) target configuration
2016-11-17 17:00:31,812 DEBUG   :         Target - Target configuration {'username': 'brendan', 'platform': 'linux', 'host': '192.168.0.1', 'ftrace': {'buffsize': 81920, 'events': ['sched_switch', 'sched_wakeup', 'sched_wakeup_new', 'cpu_frequency']}, 'board': 'juno', 'modules': ['bl', 'cpufreq'], 'password': 'test0000', 'tools': ['trace-cmd', 'perf'], 'results_dir': 'ExecutorExample'}
2016-11-17 17:00:31,813 INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'hwmon']
2016-11-17 17:00:31,814 INFO    :         Target - Connecting linux target:
2016-11-17 17:00:31,815 INFO    :         Target -   username : brendan
2016-11-17 17:00:31,816 INFO    :         Target -       host : 192.168.0.1
2016-11-17 17:00:31,817 INFO    :         Target -   password : 
2016-11-17 17:00:31,819 INFO

# Tests Configuration

In [4]:
my_tests_conf = {

    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "base",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "NO_ENERGY_AWARE",  # Disable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
        {
            "tag"            : "eas",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "ENERGY_AWARE",     # Enable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
    ],
    
    # Workloads to run (on each platform configuration)
    "wloads" : {
        # Run hackbench with 1 group using pipes
        "perf" : {
            "type" : "perf_bench",
            "conf" : {
                "class" : "messaging",
                "params" : {
                    "group" :    1,
                    "loop"  :   10,
                    "pipe"  : True,
                    "thread": True,
                }
            }
        },
        # Run a 20% duty-cycle periodic task
        "rta" : {
            "type" : "rt-app",
            "loadref" : "big",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "p20" : {
                        "kind"   : "Periodic",
                        "params" : {
                            "duty_cycle_pct" : 20,
                         },
                    },
                },
            },
        },
    },
    
    # Number of iterations for each workload
    "iterations" : 1,
}

# Tests execution

In [5]:
from executor import Executor

executor = Executor(env, my_tests_conf)

05:01:07  INFO    :         Target - Loading custom (inline) test configuration
05:01:07  INFO    : 
05:01:07  INFO    : ################################################################################
05:01:07  INFO    :       Executor - Experiments configuration
05:01:07  INFO    : ################################################################################
05:01:07  INFO    :       Executor - Configured to run:
05:01:07  INFO    :       Executor -     2 target configurations:
05:01:07  INFO    :       Executor -       base, eas
05:01:07  INFO    :       Executor -     2 workloads (1 iterations each)
05:01:07  INFO    :       Executor -       rta, perf
05:01:07  INFO    :       Executor - Total: 4 experiments
05:01:07  INFO    :       Executor - Results will be collected under:
05:01:07  INFO    :       Executor -       /home/bjackman/sources/lisa/results/ExecutorExample


In [6]:
executor.run()

05:01:07  INFO    : 
05:01:07  INFO    : ################################################################################
05:01:07  INFO    :       Executor - Experiments execution
05:01:07  INFO    : ################################################################################
05:01:07  INFO    : 
05:01:07  INFO    : ================================================================================
05:01:07  INFO    :   TargetConfig - configuring target for [base] experiments
2016-11-17 17:01:07,401 DEBUG   : sudo -- sh -c '[ -d /home/brendan/devlib-target/run_dir ] || mkdir /home/brendan/devlib-target/run_dir'
2016-11-17 17:01:08,306 DEBUG   : sudo -- sh -c 'grep schedtest /proc/mounts ||   mount -t tmpfs -o size=1024m schedtest /home/brendan/devlib-target/run_dir'
05:01:09  INFO    :  SchedFeatures - Set scheduler feature: NO_ENERGY_AWARE
2016-11-17 17:01:09,212 DEBUG   : sudo -- sh -c 'echo NO_ENERGY_AWARE > /sys/kernel/debug/sched_features'
05:01:10  INFO    :        CPUFreq - Co

In [7]:
!tree {executor.te.res_dir}

/home/bjackman/sources/lisa/results/ExecutorExample
├── perf_bench_messaging:base:perf
│   ├── 1
│   │   ├── energy.json
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── perf_bench_messaging:eas:perf
│   ├── 1
│   │   ├── energy.json
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:base:rta
│   ├── 1
│   │   ├── energy.json
│   │   ├── output.log
│   │   ├── rta_00.json
│   │   ├── rt-app-task_p200-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
└── rtapp:eas:rta
    ├── 1
    │   ├── energy.json
    │   ├── output.log
    │   ├── rta_00.json
    │   ├── rt-app-task_p200-0.log
    │   └── trace.dat
    ├── kernel.config
    ├── kernel.version
    └── platform.json

8 directories, 30 files
